In [4]:
import json

def jprint(obj):
    print(json.dumps(obj, indent=2))


In [5]:
def calculator(expression: str, **kwargs) -> str:
    """Evaluates a single line of Python math expression. No imports or variables allowed.
    
    Args:
        expression: A mathematical expression using only numbers and basic operators (+,-,*,/,**,())
        
    Returns:
        The result of the calculation or an error message
        
    Examples:
        "2 + 2" -> "4"
        "3 * (17 + 4)" -> "63"
        "100 / 5" -> "20.0"
    """
    allowed = set("0123456789+-*/.() ")
    if not all(c in allowed for c in expression):
        return "Error: Invalid characters in expression"
    
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}" 

In [25]:
import inspect
from typing import Any, Callable, Dict
from textwrap import dedent

def infer_schema_from_function(func: Callable) -> Dict[str, Any]:
    """Infers a tool schema from a function's signature and docstring."""
    sig = inspect.signature(func)
    doc = dedent(inspect.getdoc(func) or "")

    # Parse docstring sections
    doc_parts = doc.split("\n\n")
    description = doc_parts[0].strip()

    # Extract examples if present
    examples = []
    for part in doc_parts:
        part = part.strip()
        if part.startswith("Examples:"):
            examples = [line.strip() for line in part.split("\n")[1:] if line.strip()]

    # Build args schema
    args = {}
    for name, param in sig.parameters.items():
        if name == "kwargs":
            continue
        param_doc = ""
        for part in doc_parts:
            if part.strip().startswith("Args:"):
                for line in part.split("\n")[1:]:
                    if line.strip().startswith(f"{name}:"):
                        param_doc = line.strip()[len(name) + 1 :].strip()

        args[name] = {
            "type": str(param.annotation.__name__ if param.annotation != inspect.Parameter.empty else "any"),
            "description": param_doc,
        }
        if param.default != inspect.Parameter.empty:
            args[name]["default"] = param.default

    return {
        "name": func.__name__,
        "description": description,
        "args": args,
        "returns": str(sig.return_annotation.__name__ if sig.return_annotation != inspect.Parameter.empty else "any"),
        "examples": examples,
    }


In [26]:
jprint(infer_schema_from_function(calculator))

{
  "name": "calculator",
  "description": "Evaluates a single line of Python math expression. No imports or variables allowed.",
  "args": {
    "expression": {
      "type": "str",
      "description": "A mathematical expression using only numbers and basic operators (+,-,*,/,**,())"
    }
  },
  "returns": "str",
  "examples": [
    "\"2 + 2\" -> \"4\"",
    "\"3 * (17 + 4)\" -> \"63\"",
    "\"100 / 5\" -> \"20.0\""
  ]
}
